In [1]:
import pandas as pd

train = pd.read_csv("train_ctrUa4K.csv")
test = pd.read_csv("test_lAUu6dG.csv")

print ("Train shape:", train.shape)
print ("Test shape:", test.shape)
train.head()

Train shape: (614, 13)
Test shape: (367, 12)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [2]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

train["is_train"] = 1
test["is_train"] = 0
test["Loan_Status"] = np.nan
data = pd.concat([train, test], sort = False)

In [3]:
data["Gender"].fillna(data["Gender"].mode()[0])
data["Married"].fillna(data["Married"].mode()[0])
data["Dependents"].fillna(data["Dependents"].mode()[0])
data["Self_Employed"].fillna(data["Self_Employed"].mode()[0])
data["Credit_History"].fillna(data["Credit_History"].mode()[0])
data["Loan_Amount_Term"].fillna(data["Loan_Amount_Term"].mode()[0])
data["LoanAmount"].fillna(data["LoanAmount"].median())

0      126.0
1      128.0
2       66.0
3      120.0
4      141.0
       ...  
362    113.0
363    115.0
364    126.0
365    158.0
366     98.0
Name: LoanAmount, Length: 981, dtype: float64

In [4]:
data["Dependents"].replace("3+", 3, inplace = True)
data["Dependents"] = data["Dependents"].astype(int)

C:\Users\Arathi\AppData\Local\Temp\ipykernel_13284\3229492103.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Dependents"].replace("3+", 3, inplace = True)


ValueError: cannot convert float NaN to integer

In [5]:
cols = ["Gender", "Married", "Education", "Self_Employed", "Property_Area"]
for col in cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [6]:
data["Loan_Status"] = data["Loan_Status"].map({"Y":1, "N":0})

In [7]:
train = data[data["is_train"] == 1].drop(["is_train"], axis = 1)
test = data[data["is_train"] == 0].drop(["is_train", "Loan_Status"], axis = 1)

In [8]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,NaN,360.0,1.0,2,1.0
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0.0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1.0
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1.0
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
X = train.drop(["Loan_ID", "Loan_Status"], axis = 1)
y = train["Loan_Status"]

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
model = RandomForestClassifier(random_state = 42, n_estimators = 100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.7642276422764228


In [14]:
X_test = test.drop(["Loan_ID"], axis = 1)

In [15]:
test_predictions = model.predict(X_test)

In [16]:
test["Loan_Status"] = ["Y" if pred == 1 else "N" for pred in test_predictions]

In [17]:
submission = test[["Loan_ID", "Loan_Status"]]
submission.to_csv("loan_submission.csv", index = False)

In [19]:
print ("Submission file 'loan_submission.csv' created!")

Submission file 'loan_submission.csv' created!
